In [74]:
import numpy as np
import pandas as pd
import datetime
from sqlalchemy import create_engine
import plotly.express as px
from database_utils import add_new_unl, add_new_dot, clean_unleashed, clean_dot, get_level_2

In [75]:
# database connection

db_password = "UnitCircle42!"
db_user = "postgres"
db_name = "dot"
endpoint = "awakedb.cre3f7yk1unp.us-west-1.rds.amazonaws.com"

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

In [76]:
## PASTE 2 FILES BELOW AND UPDATE ALL TABLES

unl_download = r"C:\Users\mikej\Downloads\SalesEnquiryList - 2024-02-17T071954.084.xlsx"  ### file download from unleashed
dot_download = r"C:\Users\mikej\Downloads\SHOP Supplier Reporting - Invoice Details Excel - 2024-02-17T071913.307.xlsx"  ### file download from dot

add_new_unl(unl_download)  ## adds new direct purchases to raw database
add_new_dot(dot_download)  ## adds new indirect purchases to raw database

clean_unleashed()          ## clean raw direct purchases and add to clean directdb
clean_dot()                ## clean raw indirect purchases and add to clean indirect db
get_level_2()              ## combine clean direct/indirect and populate level_2 db  --  dot invisible

OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s]
[parameters: {'name': 'unleashed_raw'}]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
### updating all_sales_data.csv in "data" directory

level_2 = pd.read_sql("SELECT * FROM level_2 WHERE date > '2020-12-31';", con = engine)
level_2 = level_2[level_2.market_segment != 'Samples']
level_2.to_csv(r"C:/Users/mikej/Desktop/cpg-sales/data/all_sales_data.csv", index=False)

In [ ]:
### bar graph should populate if all is well

# level_2 = pd.read_sql('SELECT * FROM level_2;', con = engine)
level_2.date = pd.to_datetime(level_2.date)

px.bar(level_2.set_index('date').groupby(pd.Grouper(freq='M'))['usd'].sum(),
        y='usd',
        template='plotly_white',
        labels={'usd':'$USD',
                'date':''},
        color='usd',
        width=900,
        text_auto=",.2s",
        height=400).update(layout_coloraxis_showscale=False).update_traces(textposition='outside')

In [ ]:
### bar graph should populate if all is well

# level_2 = pd.read_sql('SELECT * FROM level_2;', con = engine)
level_2.date = pd.to_datetime(level_2.date)

px.bar(level_2[level_2.date>'2024-01-31'].set_index('date').groupby(pd.Grouper(freq='d'))['usd'].sum(),
        y='usd',
        template='plotly_white',
        labels={'usd':'$USD',
                'date':''},
        color='usd',
        width=900,
        color_continuous_scale='Oranges',
        text_auto=",.2s",
        height=400).update(layout_coloraxis_showscale=False).update_traces(textposition='outside')

In [70]:
# MAKE LEVEL_2 INTO TABLEU PREP OUTPUT

In [71]:
# load in level_2 and check columns

level_2 = pd.read_sql("SELECT * FROM level_2 WHERE date > '2020-12-31';", con = engine)

tab = level_2.copy().drop(columns=['qty','cad','month','year'])
tab.columns

# fix columns to mimic tableau download data output

tab.columns = ['Invoice Date', 'Sale Origin', 'Market Segment', 'Parent Customer', 'Customer', 'Item Full Description', 'Dollars', ]

# add vistar retail y/n column

tab['Vistar Retail'] = np.where(tab['Parent Customer'] == 'Vistar Retail', str('Yes'), str('No'))

tab.to_csv(r"C:\Users\mikej\Desktop\cpg-sales\data\all_sales_data_tableau.csv", index=False)

In [72]:
# make unleashed raw tableau-ready (level_1)
# load in level_1 and check columns

level_1 = pd.read_sql("SELECT * FROM unleashed_clean WHERE completed_date > '2020-12-31';", con = engine)


# load in level_1 and check columns

tab = level_1.copy().drop(columns=['month','year'])

# fix columns to mimic tableau download data output

tab.columns = ['Customer Name', 'Invoice Date','Item Full Description', 'Quantity','Sub Total', 'Dollars', 'Table','Market Segment', 'Parent Customer']

tab['Table'] = 'Unleashed'

# tab.to_csv(r"C:\Users\mikej\Desktop\cpg-sales\data\level_1_tableau.csv", index=False)

In [73]:
# l1 = pd.read_sql("SELECT * FROM unleashed_raw WHERE completed_date > '2022-12-31';", con = engine)
px.scatter(l1[l1.customer_name=='DOT Foods, Inc.'][['completed_date','sub_total']].set_index('completed_date').sort_index())

In [17]:
px.bar(pd.DataFrame(level_1.groupby('parent_customer')['usd'].sum()))